## Encountered Obstacles
- Selecting features
- The need to a massive amount of RAM capacity to merge the csv files into
- I am thinking of applying the feature selection only on df_application_train, the rest of features in the other csv files are to be selected manually.
- As there are too many csv files and all of them contain too many records, I have decided to apply feature selection only on df_application_train, manually select some features from the rest of files, and finally merge everything into each other. 
- My goal is to merge all the columns of all dataframes with the "TARGET" column exsiting in df_application_train and apply the correlation matrix to find `indepenedently` how "TARGET" is influenced by the features of this table alone
- The df_application_train was chosen to be primary as it the only df that conatins the column 'TARGET'
- To go about the data accurately, clinically, and methodically I decided that the best way to select should be through the confussion matrix insteat of my presonal judgment.
- After i have applied the feature selection methods of mine and generated totally new dataframes: df_train and df_test, it turns out there was no real difference between them and the original train and test dataframes. 

In [1]:
# !pip install pandas matplotlib -q
# !pip install numpy -q
# !pip install dask[dataframe] -q
# !pip install "dask[diagnostics]" -qprint(df)print(df)

In [2]:
import pandas as pd

In [3]:
# df_description = pd.read_csv("./data/HomeCredit_columns_description.csv")
# df_sample_submission = pd.read_csv("/kaggle/input/home-credit-default-risk/home-credit-default-risk/sample_submission.csv")
df_application_train = pd.read_csv("/kaggle/input/home-credit-default-risk/home-credit-default-risk/application_train.csv")
df_bureau_balance = pd.read_csv("/kaggle/input/home-credit-default-risk/bureau_balance.csv")
df_bureau = pd.read_csv("/kaggle/input/home-credit-default-risk/bureau.csv")
df_credit_card_balance = pd.read_csv("/kaggle/input/home-credit-default-risk/credit_card_balance.csv")
df_installments_payments = pd.read_csv("/kaggle/input/home-credit-default-risk/installments_payments.csv")
df_POS_CASH_balance = pd.read_csv("/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv")
df_previous_application = pd.read_csv("/kaggle/input/home-credit-default-risk/previous_application.csv")

In [4]:
df_application_test = pd.read_csv("/kaggle/input/home-credit-default-risk/home-credit-default-risk/application_test.csv")

In [5]:
# Drop columns with all NaN values
df_application_train = df_application_train.dropna(axis=1, how='all')
    
# Drop columns with more than 95% NaN values
threshold = len(df_application_train) * 0.95
df_application_train = df_application_train.dropna(axis=1, thresh=threshold)
    
# Drop NaN values
df_application_train = df_application_train.dropna()
        
# One-hot encode categorical variables
df_application_train = pd.get_dummies(df_application_train)

# Calculate the correlation matrix
corr_matrix = df_application_train.corr()
    
# Get the absolute correlation values with the target variable
important_features = corr_matrix['TARGET'].abs().sort_values(ascending=False)
    
# Filter features with relevance value higher than 0.08
selected_features = important_features[important_features > 0.03]

selected_feature_names = selected_features.index.tolist()

selected_feature_names.append("SK_ID_CURR")
        
# Create a new dataframe with the selected features
df_application_train_selected_features = df_application_train[selected_feature_names]

(304531, 159)


In [6]:
df_application_train_selected_features.shape

(304531, 28)

In [7]:
df_application_train.shape

(304531, 159)

In [8]:
# Drop columns with all NaN values
df_application_test = df_application_test.dropna(axis=1, how='all')
    
# Drop columns with more than 95% NaN values
threshold = len(df_application_test) * 0.95
df_application_test = df_application_test.dropna(axis=1, thresh=threshold)
    
# Drop NaN values
df_application_test = df_application_test.dropna()
        
# One-hot encode categorical variables
df_application_test = pd.get_dummies(df_application_test)

selected_feature_names.remove("TARGET")

df_application_test_selected_features = df_application_test[selected_feature_names]

print(df_application_test_selected_features.shape)

(47772, 27)


In [9]:
# List of dataframes to process
dataframes = [
    df_bureau,
    df_bureau_balance,
    df_credit_card_balance,
    df_installments_payments,
    df_POS_CASH_balance,
    df_previous_application,
]

In [10]:
def process_dataframe_train(df, merged_df):
    # Check if 'SK_ID_CURR' column exists in the dataframe
    if 'SK_ID_CURR' not in df.columns:
        return merged_df
    
    # Drop columns with all NaN values
    df = df.dropna(axis=1, how='all')
    
    # Drop columns with more than 95% NaN values
    threshold = len(df) * 0.95
    df = df.dropna(axis=1, thresh=threshold)
    
    # Drop NaN values
    df = df.dropna()
    
    df = df.drop_duplicates(subset=['SK_ID_CURR'])
        
    # One-hot encode categorical variables
    df = pd.get_dummies(df)
    
    df = pd.merge(df, df_application_train_selected_features[['SK_ID_CURR', 'TARGET']], on='SK_ID_CURR', how='left')
    
    # Calculate the correlation matrix
    corr_matrix = df.corr()
    
    # Get the absolute correlation values with the target variable
    important_features = corr_matrix['TARGET'].abs().sort_values(ascending=False)
    
    # Filter features with relevance value higher than 0.03
    selected_features = important_features[important_features > 0.06]
        
    # Extract the feature names
    selected_feature_names = selected_features.index.tolist()
    
    # Ensure 'SK_ID_CURR' is included in the selected features
    if 'SK_ID_CURR' not in selected_feature_names:
        selected_feature_names.append('SK_ID_CURR')
        
    selected_feature_names.remove("TARGET")
    selected_feature_names = list(set(selected_feature_names))
        
    # Create a new dataframe with the selected features
    df_selected_features = df[selected_feature_names]
    
    # Merge with the existing merged_df on 'SK_ID_CURR'
    merged_df = pd.merge(merged_df, df_selected_features, on='SK_ID_CURR', how='left')
    
    return merged_df

In [11]:
def process_dataframe_test(df, merged_df, df_train):
    # Check if 'SK_ID_CURR' column exists in the dataframe
    if 'SK_ID_CURR' not in df.columns:
        return merged_df
    
    # Drop columns with all NaN values
    df = df.dropna(axis=1, how='all')
    
    # Drop columns with more than 95% NaN values
    threshold = len(df) * 0.95
    df = df.dropna(axis=1, thresh=threshold)
    
    # Drop NaN values
    df = df.dropna()
    
    df = df.drop_duplicates(subset=['SK_ID_CURR'])
        
    # One-hot encode categorical variables
    df = pd.get_dummies(df)
    
    selected_feature_names = df_train.columns.tolist()
    
    # Ensure 'SK_ID_CURR' is included in the selected features
    if 'SK_ID_CURR' not in selected_feature_names:
        selected_feature_names.append('SK_ID_CURR')
        
    selected_feature_names.remove("TARGET")
    selected_feature_names = list(set(selected_feature_names))
    
    intersection_columns = list(set(df.columns) & set(selected_feature_names))
        
    # Create a new dataframe with the selected features
    df_selected_features = df[selected_feature_names]
    
    # Merge with the existing merged_df on 'SK_ID_CURR'
    merged_df = pd.merge(merged_df, df_selected_features, on='SK_ID_CURR', how='left')
    
    return df

In [12]:
# Initialize the merged dataframe with df_application_train_not_na_onehot
df_train = df_application_train_selected_features.copy()

# Process each dataframe and merge
for df in dataframes:
    df_train = process_dataframe_train(df, df_train)

In [17]:
len(df_train.columns)

33

In [14]:
# Initialize the merged dataframe with df_application_train_not_na_onehot
df_test = df_application_test_selected_features.copy()

# Process each dataframe and merge
for df in dataframes:
    df_test = process_dataframe_test(df, df_test, df_application_train)

In [16]:
len(df_test.columns)

32